## Here we show how we decided on an activation function

In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
mpl.use("TkAgg")

from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import h5py
import mne
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, LSTM, RepeatVector, Reshape, Dropout, Conv1D, UpSampling1D, Bidirectional

from os import walk, listdir


%matplotlib inline
data_path = "../../data/train/"
clear_data_path = "/media/valbub/Docs/data/train/"
raw_data_path = "../../data/resting_state/"

Using TensorFlow backend.


In [2]:
class AutoEncoder(object):
    def __init__(self, 
             input_dim = (5, 58), 
             encoded_dim = (1, 58), 
             loss="mse", 
             optimizer="adadelta", 
             activation=("relu", "sigmoid", "tanh", "elu"),
             act_idx=(0, 0),
             kernel = 3):
        
            self.input_dim = input_dim #(50, 61) or (1, 61)
            self.encoded_dim = encoded_dim #(1, 1000) or (1, 60)
    
    def prepare_clear_data(self, data_path, limit=2):
        train_eeg_dir = data_path
        train_eeg_names = [x for x in listdir(train_eeg_dir) 
                         if x[-3:] == ".h5"]
        data = np.zeros((0, 5, 58))

        flag = 0
        for eeg_name in train_eeg_names:
            if flag == limit:
                break
            flag += 1
            h5_file = h5py.File(train_eeg_dir + eeg_name, 'r')
            a_group_key = list(h5_file.keys())[0]
            eeg_data = np.array(h5_file[a_group_key]).T
            batches = np.array(self._getBatches(eeg_data, batch_size=self.input_dim[0]))
            data = np.concatenate((data, batches), axis=0)
        return data
    
    def prepare_raw_data(self, data_path, limit=2):
        def preparefile(file_path):
            raw = np.array(mne.io.read_raw_brainvision(file_path, preload=True).to_data_frame())
            batches = np.array(self._getBatches(raw, batch_size=self.input_dim[0]))
            del raw
            return batches
        files = []
        data = []
        for elem in walk(data_path):
            for file in elem[-1]:
                if file[-4:] == "vhdr":
                    files.append(file)
        data = np.ndarray(shape=(0, self.input_dim[0], self.input_dim[1]))
        flag = 0
        for file in files:
            file_name = data_path + file
            if flag == limit:
                break
            batches =  preparefile(file_name)
            data = np.concatenate((data, batches), axis=0)
            flag += 1
        return data
    
    def fit(self, X_train, epochs=50):
        X_scaled = self.scaler.fit_transform(X_train)
        self.autoencoder.fit(X_scaled, X_scaled, epochs = epochs)
    
    def encode(self, df):
        return self._predict(df, self.encoder, self.input_dim[0])
    
    def decode(self, df):
        return self._predict(df, self.decoder, self.encoded_dim[1])
    
    def run(self, df):
        return self._predict(df, self.autoencoder, self.input_dim[0])
    
    def save(self, path, part="autoencoder"):
        if part == "encoder":
            self.encoder.save(path)
        elif part == "decoder":
            self.decoder.save(path)
        elif part == "autoencoder":
            self.autoencoder.save(path)
        elif part == "model":
            self.save(path)
        pass
    
    def load(self, path, part="autoencoder"):
        if part == "encoder":
            self.encoder = keras.models.load_model(path)
        elif part == "decoder":
            self.decoder = keras.models.load_model(path)
        elif part == "autoencoder":
            self.autoencoder = keras.models.load_model(path)
        elif part == "model":
            self = keras.models.load_model(path)
        pass
    

    def _predict(self, df, model, batch_size):
        batches = self.scaler.transform(df)
        batches = tuple(self._predictBatch(batch.reshape((1, *batch.shape)), model) for batch in batches)
        batches = self._concatBatches(batches) 
        return self.scaler.reverse_transform(batches)
    
    def _predictBatch(self, batch, model):
        return model.predict(batch)
    
    def _getBatches(self, arr, batch_size, axis=0):
        n_batches = arr.shape[axis] // batch_size
        return np.array_split(arr, n_batches, axis=axis)
    
    def _concatBatches(self, batches, axis=0):
        return np.concatenate(batches, axis=axis)



In [3]:
class AU_Stage_1(AutoEncoder):
        def __init__(self, 
                 input_dim = (5, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("relu", "sigmoid", "tanh", "elu"),
                 act_idx=(0, 0),
                 kernel = 3):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._flat = Flatten()(self._inputs)
            self._encoded = Dense(encoded_dim[0]*encoded_dim[1], activation=activation[act_idx[0]])(self._flat)

            #Decoder
            self._encoded_inputs = Input(shape=(encoded_dim[0]*encoded_dim[1],))
            self._flat_decoded = Dense(input_dim[0]*input_dim[1], activation=activation[act_idx[1]])(self._encoded_inputs)
            self._decoded = Reshape(input_dim)(self._flat_decoded)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [4]:
au = AutoEncoder()
data_set = au.prepare_clear_data(clear_data_path, limit=2)
data_set.shape

(240990, 5, 58)

In [5]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_set, random_state=42, test_size=0.3)

In [6]:
train_data.shape, test_data.shape

((168693, 5, 58), (72297, 5, 58))

In [7]:
import sklearn.metrics

params = ("relu", "sigmoid", "tanh", "elu")
history = []

for i in range(4):
    for j in range(4):
        parameters = (i, j)
        au_model = AU_Stage_1(act_idx=parameters)
        au_model.fit(train_data, epochs=50)

        pr = au_model._predict(test_data, au_model.autoencoder, batch_size=5)
        ds = np.concatenate(test_data)
        p = np.concatenate(pr)
        history.append({'parameters': (params[i], params[j]), 'r2': sklearn.metrics.r2_score(ds, p), 'MSE': sklearn.metrics.mean_squared_error(ds, p), 'MAE': sklearn.metrics.mean_absolute_error(ds, p)})
        print(params[i], params[j], ':')
        print(sklearn.metrics.r2_score(ds, p))
        print(sklearn.metrics.mean_absolute_error(ds, p))
        print(sklearn.metrics.mean_squared_error(ds, p))
        print()

Epoch 1/50
168693/168693 [==============================] - 7s 44us/step - loss: 0.0767
Epoch 2/50
168693/168693 [==============================] - 7s 41us/step - loss: 0.0661
Epoch 3/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0648
Epoch 4/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0642
Epoch 5/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0639
Epoch 6/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0632
Epoch 7/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0629
Epoch 8/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0624
Epoch 9/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0619
Epoch 10/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0613
Epoch 11/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0613
Epoch 12/50
168693/168693 [===

168693/168693 [==============================] - 8s 50us/step - loss: 2.8779e-04
Epoch 41/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.8400e-04
Epoch 42/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.8032e-04
Epoch 43/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.7677e-04
Epoch 44/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.7336e-04
Epoch 45/50
168693/168693 [==============================] - 8s 49us/step - loss: 2.7005e-04
Epoch 46/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.6687e-04
Epoch 47/50
168693/168693 [==============================] - 9s 52us/step - loss: 2.6380e-04
Epoch 48/50
168693/168693 [==============================] - 8s 50us/step - loss: 2.6087e-04
Epoch 49/50
168693/168693 [==============================] - 8s 49us/step - loss: 2.5804e-04
Epoch 50/50
168693/168693 [==============================] - 8s 50us/step - loss: 

168693/168693 [==============================] - 7s 39us/step - loss: 2.0728e-04
Epoch 27/50
168693/168693 [==============================] - 7s 40us/step - loss: 2.0382e-04
Epoch 28/50
168693/168693 [==============================] - 7s 39us/step - loss: 2.0048e-04
Epoch 29/50
168693/168693 [==============================] - 6s 38us/step - loss: 1.9732e-04
Epoch 30/50
168693/168693 [==============================] - 6s 38us/step - loss: 1.9434e-04
Epoch 31/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.9147e-04
Epoch 32/50
168693/168693 [==============================] - 6s 38us/step - loss: 1.8874e-04
Epoch 33/50
168693/168693 [==============================] - 6s 38us/step - loss: 1.8614e-04
Epoch 34/50
168693/168693 [==============================] - 6s 38us/step - loss: 1.8363e-04
Epoch 35/50
168693/168693 [==============================] - 7s 40us/step - loss: 1.8124e-04
Epoch 36/50
168693/168693 [==============================] - 7s 40us/step - loss: 

168693/168693 [==============================] - 6s 37us/step - loss: 8.8928e-04
Epoch 16/50
168693/168693 [==============================] - 6s 37us/step - loss: 8.7930e-04
Epoch 17/50
168693/168693 [==============================] - 6s 38us/step - loss: 8.7122e-04
Epoch 18/50
168693/168693 [==============================] - 7s 40us/step - loss: 8.6403e-04
Epoch 19/50
168693/168693 [==============================] - 6s 38us/step - loss: 8.5723e-04
Epoch 20/50
168693/168693 [==============================] - 7s 39us/step - loss: 8.5052e-04
Epoch 21/50
168693/168693 [==============================] - 6s 38us/step - loss: 8.4380e-04
Epoch 22/50
168693/168693 [==============================] - 7s 40us/step - loss: 8.3695e-04
Epoch 23/50
168693/168693 [==============================] - 6s 37us/step - loss: 8.2994e-04
Epoch 24/50
168693/168693 [==============================] - 6s 38us/step - loss: 8.2269e-04
Epoch 25/50
168693/168693 [==============================] - 7s 40us/step - loss: 

168693/168693 [==============================] - 7s 40us/step - loss: 0.0031
Epoch 2/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0013
Epoch 3/50
168693/168693 [==============================] - 7s 39us/step - loss: 8.8535e-04
Epoch 4/50
168693/168693 [==============================] - 7s 39us/step - loss: 8.0667e-04
Epoch 5/50
168693/168693 [==============================] - 7s 39us/step - loss: 7.6501e-04
Epoch 6/50
168693/168693 [==============================] - 7s 39us/step - loss: 7.2516e-04
Epoch 7/50
168693/168693 [==============================] - 7s 39us/step - loss: 6.8340e-04
Epoch 8/50
168693/168693 [==============================] - 7s 39us/step - loss: 6.4102e-04
Epoch 9/50
168693/168693 [==============================] - 7s 39us/step - loss: 6.0026e-04
Epoch 10/50
168693/168693 [==============================] - 7s 39us/step - loss: 5.6278e-04
Epoch 11/50
168693/168693 [==============================] - 7s 39us/step - loss: 5.2936e-04
Epoch

168693/168693 [==============================] - 6s 38us/step - loss: 0.0203
Epoch 41/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0203
Epoch 42/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0203
Epoch 43/50
168693/168693 [==============================] - 6s 38us/step - loss: 0.0203
Epoch 44/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 45/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 46/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 47/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 48/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 49/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0203
Epoch 50/50
168693/168693 [==============================] - 7s 39us/step - loss: 0.0198
tanh relu :
-9.845296136709242
9.

168693/168693 [==============================] - 7s 39us/step - loss: 2.0553e-04
Epoch 28/50
168693/168693 [==============================] - 7s 39us/step - loss: 2.0282e-04
Epoch 29/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.9723e-04
Epoch 30/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.9408e-04
Epoch 31/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.9091e-04
Epoch 32/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.8680e-04
Epoch 33/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.8429e-04
Epoch 34/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.8011e-04
Epoch 35/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.7700e-04
Epoch 36/50
168693/168693 [==============================] - 7s 39us/step - loss: 1.7616e-04
Epoch 37/50
168693/168693 [==============================] - 7s 39us/step - loss: 

168693/168693 [==============================] - 7s 39us/step - loss: 0.0468
Epoch 15/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 16/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 17/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 18/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 19/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 20/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 21/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 22/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 23/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 24/50
168693/168693 [==============================] - 7s 40us/step - loss: 0.0468
Epoch 25/50
168693/168693 [======

168693/168693 [==============================] - 7s 39us/step - loss: 7.6882e-04
Epoch 3/50
168693/168693 [==============================] - 7s 39us/step - loss: 6.4797e-04
Epoch 4/50
168693/168693 [==============================] - 7s 39us/step - loss: 5.6004e-04
Epoch 5/50
168693/168693 [==============================] - 7s 39us/step - loss: 4.8929e-04
Epoch 6/50
168693/168693 [==============================] - 7s 39us/step - loss: 4.3509e-04
Epoch 7/50
168693/168693 [==============================] - 7s 40us/step - loss: 3.9366e-04
Epoch 8/50
168693/168693 [==============================] - 7s 40us/step - loss: 3.6176e-04
Epoch 9/50
168693/168693 [==============================] - 7s 39us/step - loss: 3.3671e-04
Epoch 10/50
168693/168693 [==============================] - 7s 39us/step - loss: 3.1660e-04
Epoch 11/50
168693/168693 [==============================] - 7s 39us/step - loss: 2.9994e-04
Epoch 12/50
168693/168693 [==============================] - 7s 40us/step - loss: 2.8583e

168693/168693 [==============================] - 7s 41us/step - loss: 1.1334e-04
Epoch 40/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.1143e-04
Epoch 41/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0962e-04
Epoch 42/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0788e-04
Epoch 43/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0621e-04
Epoch 44/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0461e-04
Epoch 45/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0306e-04
Epoch 46/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0159e-04
Epoch 47/50
168693/168693 [==============================] - 7s 41us/step - loss: 1.0017e-04
Epoch 48/50
168693/168693 [==============================] - 7s 41us/step - loss: 9.8789e-05
Epoch 49/50
168693/168693 [==============================] - 7s 41us/step - loss: 

In [8]:
history.sort(key=lambda x: -x['r2'])

for i in history:
    print(i['parameters'][0], i['parameters'][1])
    print('r2:', i['r2'])
    print('MSE', i['MSE'])
    print('MAE', i['MAE'])
    print()

elu elu
r2: 0.9371156454413447
MSE 1.8814306897827716e-11
MAE 1.474374763281842e-06

tanh elu
r2: 0.9338327258339739
MSE 1.8557164024302876e-11
MAE 1.5058872566288197e-06

elu tanh
r2: 0.9212061578208145
MSE 2.2908492664583917e-11
MAE 1.6411857967597822e-06

tanh tanh
r2: 0.915688069804538
MSE 2.2319780077917893e-11
MAE 1.7014348071851304e-06

relu elu
r2: 0.9082193482552848
MSE 2.47600926122255e-11
MAE 1.7798282020151449e-06

relu tanh
r2: 0.900579905975533
MSE 2.976359261850311e-11
MAE 1.840383174940357e-06

elu sigmoid
r2: 0.8939826458676104
MSE 2.2183962765449278e-11
MAE 1.9205516120347386e-06

sigmoid elu
r2: 0.8939152729773014
MSE 2.361341577826725e-11
MAE 1.8992017267869504e-06

tanh sigmoid
r2: 0.8831951230922706
MSE 2.4577994766743174e-11
MAE 1.9971123567683138e-06

relu sigmoid
r2: 0.8804104092845612
MSE 2.3040027991414168e-11
MAE 2.0454390713001816e-06

sigmoid tanh
r2: 0.8767393739862314
MSE 2.387098556886277e-11
MAE 2.0432837025837352e-06

sigmoid sigmoid
r2: 0.77075245102